# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,20.91,79,100,2.12,US,1689927113
1,1,magadan,59.5667,150.8000,23.63,76,17,4.04,RU,1689927114
2,2,monticello,45.3055,-93.7941,16.79,96,75,0.00,US,1689927114
3,3,lincheng,19.9078,109.6858,31.76,68,94,5.36,CN,1689927114
4,4,lodeynoye pole,60.7260,33.5531,17.65,66,100,5.61,RU,1689927114
5,5,saldanha,-33.0117,17.9442,11.01,52,54,3.63,ZA,1689927114
6,6,qaqortoq,60.7167,-46.0333,5.32,97,100,4.91,GL,1689927114
7,7,iqaluit,63.7506,-68.5145,3.85,100,75,1.54,CA,1689927115
8,8,waitangi,-43.9535,-176.5597,9.34,94,94,2.68,NZ,1689927115
9,9,st. john's,47.5649,-52.7093,18.97,92,95,2.24,CA,1689926931


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
map_data = city_data_df.hvplot.points("Lng","Lat",size="Humidity",color="City")

# Display the map
map_data

:Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Filter at per condition requirements:
filtered_data_df = city_data_df.loc [(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]
filtered_data_df = filtered_data_df.dropna()
filtered_data_df = filtered_data_df.loc [(city_data_df["Wind Speed"] < 4.5)]     
filtered_data_df = filtered_data_df.loc [(city_data_df["Cloudiness"] == 0)]
filtered_data_df = filtered_data_df.dropna()

# Display sample data
filtered_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
198,198,hamilton,39.1834,-84.5333,21.48,91,0,3.09,US,1689927155
249,249,itaituba,-4.2761,-55.9836,21.24,66,0,0.16,BR,1689927165
317,317,castelo do piaui,-5.3222,-41.5525,21.79,59,0,4.11,BR,1689927179
332,332,laguna,38.4210,-121.4238,25.99,61,0,2.06,US,1689927182
354,354,newman,37.3138,-121.0208,24.13,72,0,1.82,US,1689927189
423,423,taro,-8.3862,115.2800,24.00,54,0,2.06,ID,1689927204
491,491,talavera de la reina,39.9635,-4.8308,21.83,41,0,3.30,ES,1689927085


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_data_df[["City", "Country", "Lat", "Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
198,hamilton,US,39.1834,-84.5333,91,
249,itaituba,BR,-4.2761,-55.9836,66,
317,castelo do piaui,BR,-5.3222,-41.5525,59,
332,laguna,US,38.4210,-121.4238,61,
354,newman,US,37.3138,-121.0208,72,
423,taro,ID,-8.3862,115.2800,54,
491,talavera de la reina,ES,39.9635,-4.8308,41,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
radius = 10000
categories = ["accommodation.hotel"]
params = {
    "categories": categories,
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows(): 
    
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    str_latitude = str(latitude)
    str_longitude = str(longitude)
    str_radius = str(radius)
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    hotel_filter = "circle:" + str_longitude + "," + str_latitude + "," + str_radius
    params["filter"] = hotel_filter
    
    hotel_bias = "proximity:" + str_longitude + "," + str_latitude
    params["bias"] = hotel_bias
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make and API request using the params dictionaty
    response = requests.get(base_url, params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hamilton - nearest hotel: North Vista Manor
itaituba - nearest hotel: TG Hotel
castelo do piaui - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
newman - nearest hotel: No hotel found
taro - nearest hotel: Padma Resort Ubud
talavera de la reina - nearest hotel: Centro Residencial Talavera


,City,Country,Lat,Lng,Humidity,Hotel Name
198,hamilton,US,39.1834,-84.5333,91,North Vista Manor
249,itaituba,BR,-4.2761,-55.9836,66,TG Hotel
317,castelo do piaui,BR,-5.3222,-41.5525,59,No hotel found
332,laguna,US,38.4210,-121.4238,61,Holiday Inn Express & Suites
354,newman,US,37.3138,-121.0208,72,No hotel found
423,taro,ID,-8.3862,115.2800,54,Padma Resort Ubud
491,talavera de la reina,ES,39.9635,-4.8308,41,Centro Residencial Talavera


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng","Lat",color="City",size="Humidity",hover_cols=["Hotel Name", "Country"])

# Display the map
hotel_map

:Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)